# RFM Managerial Segmentation
Manegerial segmentation is simple and based on rules than ML or Statistiacal models.

![Rules](rfm-seg-rules.png)

In [1]:
import java.util.concurrent.TimeUnit
import scala.collection.mutable.ListBuffer
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

In [2]:
val schema = StructType(
                List(
                    StructField("customer_id", StringType, false),
                    StructField("purchase_amount", DoubleType, false),
                    StructField("date_of_purchase", DateType, false)
                )
            )
val data = spark.read
                .option("sep", "\t")
                .option("mode","FAILFAST")
                .option("dateFormat","YYYY-MM-dd")
                .schema(schema)
                .csv("../../data/foundation-marketing-analytics/purchases.txt")
                .toDF

schema = StructType(StructField(customer_id,StringType,false), StructField(purchase_amount,DoubleType,false), StructField(date_of_purchase,DateType,false))
data = [customer_id: string, purchase_amount: double ... 1 more field]


[customer_id: string, purchase_amount: double ... 1 more field]

In [3]:
val enriched1 = data
                    .withColumn("end_date", lit("2016-01-01"))
                    .withColumn("year_of_purchase", year($"date_of_purchase"))
                    .withColumn("days_since", datediff($"end_date", $"date_of_purchase"))
enriched1.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- purchase_amount: double (nullable = true)
 |-- date_of_purchase: date (nullable = true)
 |-- end_date: string (nullable = false)
 |-- year_of_purchase: integer (nullable = true)
 |-- days_since: integer (nullable = true)



enriched1 = [customer_id: string, purchase_amount: double ... 4 more fields]


[customer_id: string, purchase_amount: double ... 4 more fields]

In [4]:
val OneYear = 365
val TwoYears = OneYear * 2
val ThreeYears = OneYear * 3

val enriched2 = enriched1
                .groupBy($"customer_id")
                .agg(
                    max($"days_since").alias("first_purchase"),
                    min($"days_since").alias("recency"),
                    count($"*").alias("frequency"),
                    avg($"purchase_amount").alias("amount"))

enriched2.filter($"customer_id".isin("10", "90")).show(5)

+-----------+--------------+-------+---------+------+
|customer_id|first_purchase|recency|frequency|amount|
+-----------+--------------+-------+---------+------+
|         90|          3783|    758|       10| 115.8|
|         10|          3829|   3829|        1|  30.0|
+-----------+--------------+-------+---------+------+



OneYear = 365
TwoYears = 730
ThreeYears = 1095
enriched2 = [customer_id: string, first_purchase: int ... 3 more fields]


[customer_id: string, first_purchase: int ... 3 more fields]